In [1]:
# import tabulate
import json
import re
import os
import pandas as pd



In [6]:
## p1 = r"(?P<misson_name>.+)_(?P<gamma>\d+(\.\d+)?)_(?P<delta>.+)_(?P<threshold>.+)_z"
## 
## file = "alpacafarm_0.75_5.0_6.0_z.jsonl"
## matcher1 = re.match(p1, file)
## misson_name = matcher1.group("misson_name")
## threshold = matcher1.group("threshold")
## 
## print(misson_name, threshold)

# g0.5 d5.0
# g0.25 d5.0
# g0.75 d5.0
# g0.5 d2.0
# g0.25 d2.0
# g0.9 d5.0

In [8]:
# load data from pred


df = pd.DataFrame(columns=["model_name", "mission_name", "mode", "gamma", "delta", "threshold", "bl_type", "z_score", "sum"])

input_dir = "./pred"
p = r"(?P<model_name>.+)_(?P<mode>old|v2|gpt|new|no)_g(?P<gamma>.+)_d(?P<delta>\d+(\.\d+)?)"
p1 = r"(?P<misson_name>[a-zA-Z_]+)_(?P<gamma>\d+(\.\d+)?)_(?P<delta>.+)_z"

num = 0
# get all files from input_dir
for subfolder in os.listdir(input_dir):
    print("subfolder is:", subfolder)
    matcher = re.match(p, subfolder)
    model_name = matcher.group("model_name")
    mode = matcher.group("mode")
    gamma = matcher.group("gamma")
    delta = matcher.group("delta")
    
    bl_type = "None"
    bl_type = (subfolder.split("_")[-1]).split(".")[0]
    
    print(bl_type)
    if bl_type != "hard":
        if "old" in subfolder:
            bl_type = "soft"
        else:
            bl_type = "None"
        
        
    print(model_name, mode, gamma, delta, bl_type)
    
    z_score_path = os.path.join(input_dir, subfolder, "z_score")
    if os.path.exists(z_score_path):
        # print("subfolder is:", subfolder)
        files = os.listdir(z_score_path)
        
        for file in files:
            # print(file)
            # read jsons
            matcher1 = re.match(p1, file)
            if matcher1:
                misson_name = matcher1.group("misson_name")
                threshold = 6.0
            else:
                threshold = file.split("_")[-2]
            
            with open(os.path.join(z_score_path, file), "r") as f:
                data = json.load(f)

            # get data
            avarage_z = data["avarage_z"]
            sum = len(data["z_score_list"])
            # wm_pred_avarage = data["wm_pred_avarage"]
            
            temp = pd.DataFrame({
                "model_name": [model_name],
                "mode": [mode],
                "mission_name": [misson_name],
                "gamma": [gamma],
                "delta":[delta],
                "threshold": [threshold],
                "bl_type": [bl_type],
                "z_score": [avarage_z],
                "sum": [sum]})
            
            df = pd.concat([df, temp], ignore_index=True)
            num += 1

df.to_csv("z_score.csv")           
print(df)
print(num)
                



subfolder is: llama2-7b-chat-4k_gpt_g0.25_d10.0
d10
llama2-7b-chat-4k gpt 0.25 10.0 None
subfolder is: llama2-7b-chat-4k_gpt_g0.1_d10.0
d10
llama2-7b-chat-4k gpt 0.1 10.0 None
subfolder is: llama2-7b-chat-4k_old_g0.25_d15.0_hard
hard
llama2-7b-chat-4k old 0.25 15.0 hard
subfolder is: llama2-7b-chat-4k_old_g0.75_d5.0_hard
hard
llama2-7b-chat-4k old 0.75 5.0 hard
subfolder is: llama2-7b-chat-4k_v2_g0.25_d2.0
d2
llama2-7b-chat-4k v2 0.25 2.0 None
subfolder is: llama2-7b-chat-4k_old_g0.5_d2.0_hard
hard
llama2-7b-chat-4k old 0.5 2.0 hard
subfolder is: llama2-7b-chat-4k_old_g0.25_d5.0_hard
hard
llama2-7b-chat-4k old 0.25 5.0 hard
subfolder is: llama2-7b-chat-4k_gpt_g0.5_d5.0
d5
llama2-7b-chat-4k gpt 0.5 5.0 None
subfolder is: llama2-7b-chat-4k_old_g0.1_d5.0
d5
llama2-7b-chat-4k old 0.1 5.0 soft
subfolder is: llama2-7b-chat-4k_v2_g0.25_d10.0
d10
llama2-7b-chat-4k v2 0.25 10.0 None
subfolder is: llama2-7b-chat-4k_v2_g0.1_d5.0
d5
llama2-7b-chat-4k v2 0.1 5.0 None
subfolder is: llama2-7b-chat-4k

In [3]:
df_sorted = df.sort_values(by="z_score", ascending=True)
df_sorted.to_csv("z_score_sorted.csv")           
print(df_sorted)

            model_name             mission_name mode gamma delta  threshold  \
255  llama2-7b-chat-4k                      lcc   no   0.5   5.0        6.0   
238  llama2-7b-chat-4k                 hotpotqa  old   0.9   5.0        6.0   
242  llama2-7b-chat-4k  konwledge_understanding  old   0.9   5.0        6.0   
218  llama2-7b-chat-4k  konwledge_understanding   v2   0.5   2.0        6.0   
161  llama2-7b-chat-4k  konwledge_understanding  old   0.9   5.0        6.0   
..                 ...                      ...  ...   ...   ...        ...   
360  llama2-7b-chat-4k               multi_news  old   0.1  10.0        6.0   
12   llama2-7b-chat-4k                    qmsum  gpt   0.1  10.0        6.0   
276  llama2-7b-chat-4k               multi_news  old   0.1   5.0        6.0   
315  llama2-7b-chat-4k               multi_news  old   0.1  10.0        6.0   
15   llama2-7b-chat-4k               multi_news  gpt   0.1  10.0        6.0   

    bl_type    z_score  sum  
255    None  -0.91561

In [12]:
### process_mutual
df = pd.DataFrame(columns=["model_name", "mission_name", "ref_mode", "det_mode", "threshold", "z_score", "sum"])

input_dir = "./mutual_detect/llama2-7b-chat-4k"

num = 0
p1 = r"(?P<misson_name>[a-zA-Z_]+)_(?P<threshold>\d+(\.\d+)?)_z"

# get all files from input_dir
for subfolder in os.listdir(input_dir):
    print("subfolder is:", subfolder)
    
    ref_mode = subfolder.split("ref_")[1]
        
    print(ref_mode)
    
    for subsubfolder in os.listdir(os.path.join(input_dir,subfolder)):
        
        det_mode = subsubfolder.split("_z")[0]
        print(det_mode)
    
        z_score_path = os.path.join(input_dir, subfolder, subsubfolder)
        if os.path.exists(z_score_path):
        # print("subfolder is:", subfolder)
            files = os.listdir(z_score_path)
        
            for file in files:
                # print(file)
                # read jsons
                matcher1 = re.match(p1, file)
                if matcher1:
                    misson_name = matcher1.group("misson_name")
                    threshold = matcher1.group("threshold")
                with open(os.path.join(z_score_path, file), "r") as f:
                    data = json.load(f)

                # get data
                avarage_z = data["avarage_z"]
                sum = len(data["z_score_list"])
                # wm_pred_avarage = data["wm_pred_avarage"]
                
                temp = pd.DataFrame({
                    "model_name": [model_name],
                    "ref_mode": [ref_mode],
                    "mission_name": [misson_name],
                    "det_mode":[det_mode],
                    "threshold": [threshold],
                    "z_score": [avarage_z],
                    "sum": [sum]})
                
                df = pd.concat([df, temp], ignore_index=True)
                num += 1

df.to_csv("mutual_z_score.csv")           
print(df)
print(num)


subfolder is: ref_old_soft_g0.1_d10.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_hard_g0.25_d5.0
old_soft_g0.25_d15.0
v2_g0.25_d15.0
subfolder is: ref_old_hard_g0.25_d5.0
old_hard_g0.25_d5.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_soft_g0.25_d15.0
v2_g0.25_d15.0
subfolder is: ref_old_soft_g0.25_d15.0
old_soft_g0.25_d15.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_hard_g0.25_d5.0
v2_g0.25_d15.0
subfolder is: ref_gpt_g0.1_d10.0
gpt_g0.1_d10.0
old_soft_g0.1_d10.0
old_hard_g0.25_d5.0
old_soft_g0.25_d15.0
v2_g0.25_d15.0
subfolder is: ref_v2_g0.25_d15.0
v2_g0.25_d15.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_hard_g0.25_d5.0
old_soft_g0.25_d15.0
            model_name             mission_name             ref_mode  \
0    llama2-7b-chat-4k               multi_news  old_soft_g0.1_d10.0   
1    llama2-7b-chat-4k               alpacafarm  old_soft_g0.1_d10.0   
2    llama2-7b-chat-4k                      lcc  old_soft_g0.1_d10.0   
3    llama2-7b-chat-4k              longform_qa  old_soft_g0.1_d10.0   
4   